In [2]:
import ee
import folium
import geemap.foliumap as geemap

import matplotlib.pyplot as plt
%matplotlib inline

# Package website
 https://geemap.org/

In [1]:
# import ee
# import geemap

# Map = geemap.Map()
# import geemap

# Map = geemap.Map()

# countries = ee.FeatureCollection("FAO/GAUL/2015/level1")
# thessalia = countries.filter(ee.Filter.eq('ADM1_NAME','Thessalia'))


# Map.centerObject(thessalia, 8)
# Map.addLayer(thessalia, {'fillColor': 'blue', 'fillOpacity': 0.5, 'color': 'blue'}, 'Thessalia')

# Map

In [15]:
import ee
import geemap


Map = geemap.Map()

countries = ee.FeatureCollection("FAO/GAUL/2015/level1")
thessalia = countries.filter(ee.Filter.eq('ADM1_NAME','Thessalia'))

# Functions
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(["B8", "B4"])
    return ndvi

def calculate_ndwi(image):
    ndvi = image.normalizedDifference(["B8", "B3"])
    return ndvi

def calculate_mndwi(image):
    ndvi = image.normalizedDifference(["B11", "B3"])
    return ndvi

def calculate_ndmi(image):
    ndvi = image.normalizedDifference(["B11", "B8"])
    return ndvi

## Before the event
sentinel2_before = ee.ImageCollection('COPERNICUS/S2')\
.filterDate('2023-08-31', '2023-09-01').filterBounds(thessalia)


# Applyinh the indices
ndvi_collection_before = sentinel2_before.map(calculate_ndvi)
ndwi_collection_before = sentinel2_before.map(calculate_ndwi)
mndwi_collection_before = sentinel2_before.map(calculate_mndwi)
ndmi_collection_before = sentinel2_before.map(calculate_ndmi)

## after the event 
sentinel2_after = ee.ImageCollection('COPERNICUS/S2')\
.filterDate('2023-09-10', '2023-09-11').filterBounds(thessalia)


# Applyinh the indices
ndvi_collection_after = sentinel2_after.map(calculate_ndvi)
ndwi_collection_after = sentinel2_after.map(calculate_ndwi)
mndwi_collection_after = sentinel2_after.map(calculate_mndwi)
ndmi_collection_after = sentinel2_after.map(calculate_ndmi)

# Calculate NDVI for each image in the collection

ndvi_vis = {"min": -1, "max": 1, "palette": ["red", "yellow", "green"]}
ndwi_vis = {"min": -1, "max": 1, "palette": ["green", "white", "blue"]}

vis_S2 = {
    'bands': ['B4', 'B3', 'B2'],  # Display the true-color bands (red, green, blue)
    'min': 0,
    'max': 3000
}

# Map.addLayer(sentinel2, vis_S2, 'Sentinel')
# Map.addLayer(ndvi_collection_before, ndvi_vis, 'NDVI')

# Map.addLayer(ndvi_collection_after, ndvi_vis, 'NDVI')

left_layer = geemap.ee_tile_layer(ndvi_collection_before, ndvi_vis, name='NDVI before')
right_layer = geemap.ee_tile_layer(ndvi_collection_after, ndvi_vis, name='NDVI after')

Map.split_map(left_layer, right_layer)
Map.setCenter(22, 38, 7)
Map


Map(center=[38, 22], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [22]:
import ee
import geemap

Map = geemap.Map()

countries = ee.FeatureCollection("FAO/GAUL/2015/level1")
thessalia = countries.filter(ee.Filter.eq('ADM1_NAME', 'Thessalia'))

# Functions
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(["B8", "B3"])
    return ndwi

# Create a binary mask image where masked areas are set to 1
mask_image = ee.Image(1).clip(thessalia).mask()

## Before the event
sentinel2_before = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate('2023-08-31', '2023-09-01').filterBounds(thessalia)

# Applying the NDWI index
ndwi_collection_before = sentinel2_before.map(calculate_ndwi)

## After the event
sentinel2_after = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate('2023-09-10', '2023-09-11').filterBounds(thessalia)

# Applying the NDWI index
ndwi_collection_after = sentinel2_after.map(calculate_ndwi)

# Update the mask of NDWI collections using the binary mask
ndwi_collection_before_masked = ndwi_collection_before.map(lambda image: image.updateMask(mask_image))
ndwi_collection_after_masked = ndwi_collection_after.map(lambda image: image.updateMask(mask_image))

ndwi_vis = {"min": -1, "max": 1, "palette": ["green", "white", "blue"]}

left_layer = geemap.ee_tile_layer(ndwi_collection_before_masked, ndwi_vis, name='NDWI before')
right_layer = geemap.ee_tile_layer(ndwi_collection_after_masked, ndwi_vis, name='NDWI after')



Map.split_map(left_layer, right_layer, left_label='Before the event',
    right_label='After the event')
Map.setCenter(22, 38, 7)
Map


Map(center=[38, 22], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [23]:
import ee
import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

# GPM V6 30 minute data around hurricane Dorian for a single day.


startDate = ee.Date('2023-09-01')
endDate = ee.Date('2023-09-06');  # Adjust as needed

# Define the date range for the period
dateRange = ee.DateRange(startDate, endDate)

dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filter(ee.Filter.date(dateRange)) \

# Select the max precipitation and mask out low precipitation values.
precipitation = dataset.select('precipitationCal').max()
mask = precipitation.gt(0.5)
precipitation = precipitation.updateMask(mask)

palette = [
  '000096','0064ff', '00b4ff', '33db80', '9beb4a',
  'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000'
]
precipitationVis = {'min': 0, 'max': 15, 'palette': palette}
Map.addLayer(precipitation, precipitationVis, 'Precipitation (mm/hr)')

Map.setCenter(22, 38, 7)

Map

Map(center=[38, 22], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…